# Web Crawler 

## 實作1: 爬取PTT八卦版的文章資訊

### 以PTT八卦版為例 : https://www.ptt.cc/bbs/Gossiping/index.html

### Step1. import 套件

In [1]:
import requests
from bs4 import BeautifulSoup

# 把Warning訊息去掉
requests.packages.urllib3.disable_warnings()

### Step2. 將網頁Get下來

In [2]:
# 設定proxy 在公司內需使用
http_proxy = "http://auhqproxy.corpnet.auo.com:8080/"
https_proxy = "http://auhqproxy.corpnet.auo.com:8080/"
ftp_proxy = "http://auhqproxy.corpnet.auo.com:8080/"

proxyDict = { 
              "http"  : http_proxy, 
              "https" : https_proxy, 
              "ftp"   : ftp_proxy
            }
            
#r = requests.get(url, proxies=proxyDict, verify=False)

In [3]:
# 要爬的網站
url = "https://www.ptt.cc/bbs/Gossiping/index.html"

### Step3. 處理PTT網站詢問是否已滿18歲的問題
1. 進入網站後會看到讓使用者點選「是否已滿18歲」按鈕

2. 可開啟瀏覽器開發者模式(F12)，並點選至network(網路)，觀察點選「已滿18歲」後，會送給伺服器之封包內容。

3. 發現封包以POST發送，並且封包內容有cookie和參數

4. 我們的程式碼可以透過POST的方式，送出cookie和特定的參數

5. 可以藉由requests的一個參數來儲存自身的cookie
   Session會將你送出的requests所收到的cookies全部儲存起來並且在發送下一次請求時送出對應的參數。

In [4]:
# 處理是否已滿18歲的問題
payload = {
    "from":"/bbs/Gossiping/index.html",
    "yes":"yes"
}

rs = requests.session()

# 送出資訊已滿18歲的資訊
# verify=False 是為了避免SSL認證
res = rs.post("https://www.ptt.cc/ask/over18", proxies=proxyDict, verify=False, data=payload)

# 將此頁面的HTML GET下來
res = rs.get(url, proxies=proxyDict, verify=False)

# 印出HTML
print(res.text)

<!DOCTYPE html>
<html>
	<head>
		<meta charset="utf-8">
		

<meta name="viewport" content="width=device-width, initial-scale=1">

<title>看板 Gossiping 文章列表 - 批踢踢實業坊</title>

<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-common.css">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-base.css" media="screen">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-custom.css">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/pushstream.css" media="screen">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-print.css" media="print">




	<script async src='/cdn-cgi/challenge-platform/h/b/scripts/invisible.js?ts=1650517200'></script></head>
    <body>
		
<div id="topbar-container">
	<div id="topbar" class="bbs-content">
		<a id="logo" href="/bbs/">批踢踢實業坊</a>
		<span>&rsaquo;</span>
		<a class="board" href="/bbs/Gossiping/index.html"><span class="board-label">看板 </span>Gossi

### Step4. 擷取各文章標題
```html=
<div class="title">	
    <a href="/bbs/Gossiping/M.1650337301.A.456.html">[新聞] 基隆無症狀、輕症 居家照護今啟動</a>
</div>
```

In [5]:
# 將網頁資料以html.parser
soup = BeautifulSoup(res.text, "html.parser")

# 取HTML標中的 <div class="title"></div> 中的<a>標籤存入title
title = soup.select("div.title a")
print(title)

[<a href="/bbs/Gossiping/M.1650519761.A.AE3.html">[問卦] 8818</a>, <a href="/bbs/Gossiping/M.1650519932.A.AFF.html">[新聞] 小玉出庭鞠躬道歉！靠換臉A片撈1338萬　</a>, <a href="/bbs/Gossiping/M.1650520012.A.DB5.html">[問卦] 小馮假使今天是腳交女  風向會變嘛= =?</a>, <a href="/bbs/Gossiping/M.1650520020.A.CAE.html">[問卦] 辦公室新員工一直喊yubi yubi = =</a>, <a href="/bbs/Gossiping/M.1645455874.A.65E.html">[公告] 八卦板板規(2022.02.21)</a>, <a href="/bbs/Gossiping/M.1648646343.A.E14.html">[協尋] 3/30大漢橋新莊往板橋機車道行車紀錄器</a>, <a href="/bbs/Gossiping/M.1649960701.A.CDB.html">Fw: [公告] Ptt 即日起關閉無加密的 telnet 連線方式</a>, <a href="/bbs/Gossiping/M.1650354748.A.607.html">[協尋] 4/15中午新竹市水田街90號車禍行車記錄器</a>, <a href="/bbs/Gossiping/M.1650418069.A.FFE.html">[公告] 小心辨別所有來源資訊 注意帳號安全 水桶</a>]


In [6]:
# 將爬下來的文章標題印出來
for s in title:
    # https://www.ptt.cc/
    print("https://www.ptt.cc/" + s["href"], s.text)

https://www.ptt.cc//bbs/Gossiping/M.1650519761.A.AE3.html [問卦] 8818
https://www.ptt.cc//bbs/Gossiping/M.1650519932.A.AFF.html [新聞] 小玉出庭鞠躬道歉！靠換臉A片撈1338萬　
https://www.ptt.cc//bbs/Gossiping/M.1650520012.A.DB5.html [問卦] 小馮假使今天是腳交女  風向會變嘛= =?
https://www.ptt.cc//bbs/Gossiping/M.1650520020.A.CAE.html [問卦] 辦公室新員工一直喊yubi yubi = =
https://www.ptt.cc//bbs/Gossiping/M.1645455874.A.65E.html [公告] 八卦板板規(2022.02.21)
https://www.ptt.cc//bbs/Gossiping/M.1648646343.A.E14.html [協尋] 3/30大漢橋新莊往板橋機車道行車紀錄器
https://www.ptt.cc//bbs/Gossiping/M.1649960701.A.CDB.html Fw: [公告] Ptt 即日起關閉無加密的 telnet 連線方式
https://www.ptt.cc//bbs/Gossiping/M.1650354748.A.607.html [協尋] 4/15中午新竹市水田街90號車禍行車記錄器
https://www.ptt.cc//bbs/Gossiping/M.1650418069.A.FFE.html [公告] 小心辨別所有來源資訊 注意帳號安全 水桶


***

## 擷取多頁PTT

In [7]:
# 要爬的網站
url = "https://www.ptt.cc/bbs/Gossiping/index.html"

for i in range(3):
    # 將此頁面的HTML GET下來
    res = rs.get(url, proxies=proxyDict, verify=False)
 
    # 將網頁資料以html.parser
    soup = BeautifulSoup(res.text, "html.parser")

    # 取HTML標中的 <div class="title"></div> 中的<a>標籤存入title
    title = soup.select("div.title a")
    
    # 上一頁按鈕的a標籤
    u = soup.select("div.btn-group.btn-group-paging a")
    
    print("此頁URL:", url)
    
    for element in title:
        print("https://www.ptt.cc/" + element["href"], element.text)
    
    print("--------------------------------------------------------------------------\n")
    
    # 組合出上一頁的網址
    url = "https://www.ptt.cc"+ u[1]["href"]
    
    


此頁URL: https://www.ptt.cc/bbs/Gossiping/index.html
https://www.ptt.cc//bbs/Gossiping/M.1650519761.A.AE3.html [問卦] 8818
https://www.ptt.cc//bbs/Gossiping/M.1650519932.A.AFF.html [新聞] 小玉出庭鞠躬道歉！靠換臉A片撈1338萬　
https://www.ptt.cc//bbs/Gossiping/M.1650520012.A.DB5.html [問卦] 小馮假使今天是腳交女  風向會變嘛= =?
https://www.ptt.cc//bbs/Gossiping/M.1650520020.A.CAE.html [問卦] 辦公室新員工一直喊yubi yubi = =
https://www.ptt.cc//bbs/Gossiping/M.1645455874.A.65E.html [公告] 八卦板板規(2022.02.21)
https://www.ptt.cc//bbs/Gossiping/M.1648646343.A.E14.html [協尋] 3/30大漢橋新莊往板橋機車道行車紀錄器
https://www.ptt.cc//bbs/Gossiping/M.1649960701.A.CDB.html Fw: [公告] Ptt 即日起關閉無加密的 telnet 連線方式
https://www.ptt.cc//bbs/Gossiping/M.1650354748.A.607.html [協尋] 4/15中午新竹市水田街90號車禍行車記錄器
https://www.ptt.cc//bbs/Gossiping/M.1650418069.A.FFE.html [公告] 小心辨別所有來源資訊 注意帳號安全 水桶
--------------------------------------------------------------------------

此頁URL: https://www.ptt.cc/bbs/Gossiping/index39247.html
https://www.ptt.cc//bbs/Gossiping/M.1650518916.A.6F4.html [問卦] 哦哦